In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [37]:
!pip install google-generativeai

In [38]:
!pip install pinecone

In [ ]:
gemini_api_key_1 = ""
gemini_api_key_2 = ""
gemini_api_key_3 = ""

In [40]:
# Input legal query to be improved
query = "Murder commited by minor"

In [41]:
import requests
import json

# Replace with your actual API key
api_key = gemini_api_key_2
url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={api_key}"



# Prepare the data for improving the query
data = {
    "contents": [
        {
            "parts": [
                {
                    "text": f"Rewrite the following legal query to maximize retrieval accuracy from the Indian Penal Code (IPC) using cosine similarity. "
                            f"Expand the query by incorporating relevant legal terms, synonyms, related offenses, and broader legal contexts. "
                            f"Ensure the language is precise and comprehensive to improve matching with relevant laws. "
                            f"Do not include specific article numbers or factual information—focus solely on enhancing the query's expressiveness. Increase the number of keywords so that cosine similarity retrieval is aided "
                            f"Provide only the improved query, nothing else.\n\nQuery:\n{query}"
                }
            ]
        }
    ]
}

# Set the headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request
response = requests.post(url, headers=headers, json=data)

# Check the response and parse the improved query
if response.status_code == 200:
    response_data = response.json()
    improved_query = response_data.get("candidates", [{}])[0].get("content", {}).get("parts", [{}])[0].get("text", "").strip()
    
    # print("Original Query:", query)
    # print("Improved Query:", improved_query)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


In [42]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

model = SentenceTransformer("nlpaueb/legal-bert-base-uncased")

user_query = improved_query

query_embedding = model.encode(user_query, convert_to_numpy=True).tolist()  # Convert to list for Pinecone




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [55]:
pc = Pinecone("pcsk_3eb3fw_R8LSeRPSZyQpcLpMsMZ6iG3LFN1DUAbbUs7WBend67X4ocTzdCbe8ovBHALB4gb")  # Replace securely
index = pc.Index("ipc")

search_results = index.query(vector=query_embedding, top_k=10, include_metadata=True) 
results_list = []

for match in search_results["matches"]:
    results_list.append({
        "Offense": match["metadata"]["Offense"],
        "Description": match["metadata"]["Description"],
        "Punishment": match["metadata"]["Punishment"]
    })


In [57]:
import requests
from bs4 import BeautifulSoup

def scrape_livelaw_article(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    article_content = ""
    article_body = soup.find('div', class_='_s30J')  # Adjusted div class
    if article_body:
        for para in article_body.find_all(text=True, recursive=False):
            article_content += para.strip() + "\n"

    return article_content if article_content else None

def scrape_toi_article(url, results_list):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return

    soup = BeautifulSoup(response.text, 'html.parser')

    first_div = soup.find_all('div', class_='crmK8')  # Adjust the class name if necessary

    for div in first_div:
        a_tags = div.find_all("a", href=True)

        for a in a_tags[:5]:
            href = a["href"]
            full_link = f"https://www.livelaw.in{href}" if href.startswith("/") else href
            
            content = scrape_livelaw_article(full_link)
            if content:
                results_list.append({"Article Heading": a.text.strip(), "Content": content})

if __name__ == "__main__":
    topic = query
    url = f"https://timesofindia.indiatimes.com/topic/{topic}/news"
    scrape_toi_article(url, results_list)


<ipython-input-57-ffcda3342e18>:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for para in article_body.find_all(text=True, recursive=False):


In [58]:
results_list

[{'Offense': 'Robber or Dacoity, with attempt to cause death or grievous hurt',
  'Description': 'Description of IPC Section 397\nAccording to section 397 of Indian penal code, If, at the time of committing robbery or dacoity, the offender uses any deadly weapon, or causes grievous hurt to any person, so attempts to cause death or grievous hurt to any person, the imprisonment with which such offender shall be punished shall not be less than seven years.\n\n\nIPC 397 in Simple Words\nIf a person commits robbery or dacoity while using a deadly weapon or causing grievous hurt, they will be punished with imprisonment of at least seven years.\n\nCited by\nGilbert Pereira vs State Of Karnataka\nDeepak at Wireless vs State Of Maharashtra\nOma at Omprakash and Anr vs State Of Tamilnadu\nAshfaq vs State Govt Of Nct Of Delhi\nDilawar Singh vs State Of Delhi',
  'Punishment': 'Rigorous Imprisonment for not less than 7 Years'},
 {'Offense': 'Lurking house-trespass or house-breaking after preparati

In [60]:
import requests
import json

# Replace with your actual API key
api_key = gemini_api_key_2
url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={api_key}"

def extract_relevant_points(query, results):
    # Prepare the request payload
    data = {
        "contents": [
            {
                "parts": [
                    {
                        "text": f"Given the following legal query, extract and arrange the most relevant points from the provided legal texts. "
                                f"Ensure the points are clear, structured, and directly address the query. "
                                f"Remove any redundant or unnecessary information.\n\n"
                                f"Query:\n{query}\n\n"
                                f"Legal Texts:\n{results_list}\n\n"
                                f"Provide the response in a structured, pointwise format under the heading 'Relevant Legal Points'."
                    }
                ]
            }
        ]
    }

    # Set headers
    headers = {
        "Content-Type": "application/json"
    }

    # Send the request
    response = requests.post(url, headers=headers, json=data)

    # Process the response
    if response.status_code == 200:
        response_data = response.json()
        generated_text = response_data.get("candidates", [{}])[0].get("content", {}).get("parts", [{}])[0].get("text", "").strip()
        return generated_text
    else:
        return f"Error: {response.status_code}\n{response.text}"

# Example usage


response_text = extract_relevant_points(query, results_list)
print(response_text)


**Relevant Legal Points**

* **Indian Penal Code (IPC) Section 302 (Murder):**  This section addresses the offense of murder.  While the provided texts don't explicitly detail the punishment for a minor committing murder under this section,  news articles illustrate that minors can be charged with murder and that sentences can range from life imprisonment (possibly without parole) to death (though death sentences are subject to review and potential commutation).  The sentencing considers mitigating and aggravating circumstances.

* **Age of the Accused:** The age of the minor committing the murder is a crucial factor influencing the legal proceedings and sentencing.  The IPC Section 305 regarding abetment of suicide by a minor indicates that the age of the accused (under 18) impacts the potential penalties.  Cases involving minors often require specialized juvenile justice procedures.

* **Sentencing Considerations:**  Sentencing for murder, even when committed by a minor, is determine